In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import ast 
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
# from nltk.stem.snowball import SnowballStemmer
# from nltk.stem.wordnet import WordNetLemmatizer
# from nltk.corpus import wordnet
# from surprise import Reader, Dataset, SVD, evaluate

import warnings; warnings.simplefilter('ignore')


In [56]:
apps = pd.read_csv("C:\\Users\\Ishaan\\Desktop\\kodikon\\apps.tsv", delimiter='\t',encoding='utf-8')
user_history = pd.read_csv("C:\\Users\\Ishaan\\Desktop\\kodikon\\user_history.tsv", delimiter='\t',encoding='utf-8')
jobs = pd.read_csv("C:\\Users\\Ishaan\\Desktop\\kodikon\\jobs.tsv", delimiter='\t',encoding='utf-8', error_bad_lines=False)
users = pd.read_csv("C:\\Users\\Ishaan\\Desktop\\kodikon\\users.tsv" ,delimiter='\t',encoding='utf-8')
test_users = pd.read_csv("C:\\Users\\Ishaan\\Desktop\\kodikon\\test_users.tsv", delimiter='\t',encoding='utf-8')


b'Skipping line 122433: expected 11 fields, saw 12\n'
b'Skipping line 602576: expected 11 fields, saw 12\n'
b'Skipping line 990950: expected 11 fields, saw 12\n'


NameError: name 'app' is not defined

In [3]:
apps_training = apps.loc[apps['Split'] == 'Train']
apps_testing = apps.loc[apps['Split'] == 'Test']
user_history_training = user_history.loc[user_history['Split'] =='Train']
user_history_testing = user_history.loc[user_history['Split'] =='Test']
users_training = users.loc[users['Split']=='Train']
users_testing = users.loc[users['Split']=='Test']

In [12]:
jobs_US = jobs.loc[jobs['Country']=='US']
jobs_US

,JobID,WindowID,Title,Description,Requirements,City,State,Country,Zip5,StartDate,EndDate
0,1,1,Security Engineer/Technical Lead,<p>Security Clearance Required:&nbsp; Top Secr...,<p>SKILL SET</p>\r<p>&nbsp;</p>\r<p>Network Se...,Washington,DC,US,20531,2012-03-07 13:17:01.643,2012-04-06 23:59:59
1,4,1,SAP Business Analyst / WM,<strong>NO Corp. to Corp resumes&nbsp;are bein...,<p><b>WHAT YOU NEED: </b></p>\r<p>Four year co...,Charlotte,NC,US,28217,2012-03-21 02:03:44.137,2012-04-20 23:59:59
2,7,1,P/T HUMAN RESOURCES ASSISTANT,<b> <b> P/T HUMAN RESOURCES ASSISTANT</b> <...,Please refer to the Job Description to view th...,Winter Park,FL,US,32792,2012-03-02 16:36:55.447,2012-04-01 23:59:59
3,8,1,Route Delivery Drivers,CITY BEVERAGES Come to work for the best in th...,Please refer to the Job Description to view th...,Orlando,FL,US,NaN,2012-03-03 09:01:10.077,2012-04-02 23:59:59
4,9,1,Housekeeping,I make sure every part of their day is magica...,Please refer to the Job Description to view th...,Orlando,FL,US,NaN,2012-03-03 09:01:11.88,2012-04-02 23:59:59
5,10,1,SALON/SPA COORDINATOR,<b> <b>— SALON/SPA COORDINATOR ...,Please refer to the Job Description to view th...,Ormond Beach,FL,US,32174,2012-03-05 14:21:50.203,2012-04-04 23:59:59
6,11,1,SUPERINTENDENT,<b> <b>SUPERINTENDENT</b> </b> —— Central...,Please refer to the Job Description to view th...,Orlando,FL,US,32801,2012-03-06 09:21:54.58,2012-04-05 23:59:59
7,12,1,ELECTRONIC PRE-PRESS PROFESSIONAL,<b> <b>ELECTRONIC PRE-PRESS PROFESSIONAL</b...,Please refer to the Job Description to view th...,Orlando,FL,US,32808,2012-03-06 11:21:53.63,2012-04-05 23:59:59
8,13,1,UTILITY LINE TRUCK OPERATOR/ DIGGER DERRICK,<b> </b> \r\n <b> <b>UTILITY LINE TRUCK...,Please refer to the Job Description to view th...,Orlando,FL,US,32801,2012-03-06 16:06:53.677,2012-04-05 23:59:59
9,14,1,CONSTRUCTION PROJECT MGR & PM TRAINEE,<b> <b>CONSTRUCTION PROJECT MGR </b> </b> \...,Please refer to the Job Description to view th...,Winter Park,FL,US,32789,2012-03-07 10:21:37.467,2012-04-06 23:59:59


In [39]:
jobs_US_base_line = jobs_US.iloc[0:10000,0:8]
jobs_US_base_line = jobs_US_base_line.dropna()
jobs_US_base_line = jobs_US_base_line.reset_index(drop=True)
jobs_US_base_line

,JobID,WindowID,Title,Description,Requirements,City,State,Country
0,1,1,Security Engineer/Technical Lead,<p>Security Clearance Required:&nbsp; Top Secr...,<p>SKILL SET</p>\r<p>&nbsp;</p>\r<p>Network Se...,Washington,DC,US
1,4,1,SAP Business Analyst / WM,<strong>NO Corp. to Corp resumes&nbsp;are bein...,<p><b>WHAT YOU NEED: </b></p>\r<p>Four year co...,Charlotte,NC,US
2,7,1,P/T HUMAN RESOURCES ASSISTANT,<b> <b> P/T HUMAN RESOURCES ASSISTANT</b> <...,Please refer to the Job Description to view th...,Winter Park,FL,US
3,8,1,Route Delivery Drivers,CITY BEVERAGES Come to work for the best in th...,Please refer to the Job Description to view th...,Orlando,FL,US
4,9,1,Housekeeping,I make sure every part of their day is magica...,Please refer to the Job Description to view th...,Orlando,FL,US
5,10,1,SALON/SPA COORDINATOR,<b> <b>— SALON/SPA COORDINATOR ...,Please refer to the Job Description to view th...,Ormond Beach,FL,US
6,11,1,SUPERINTENDENT,<b> <b>SUPERINTENDENT</b> </b> —— Central...,Please refer to the Job Description to view th...,Orlando,FL,US
7,12,1,ELECTRONIC PRE-PRESS PROFESSIONAL,<b> <b>ELECTRONIC PRE-PRESS PROFESSIONAL</b...,Please refer to the Job Description to view th...,Orlando,FL,US
8,13,1,UTILITY LINE TRUCK OPERATOR/ DIGGER DERRICK,<b> </b> \r\n <b> <b>UTILITY LINE TRUCK...,Please refer to the Job Description to view th...,Orlando,FL,US
9,14,1,CONSTRUCTION PROJECT MGR & PM TRAINEE,<b> <b>CONSTRUCTION PROJECT MGR </b> </b> \...,Please refer to the Job Description to view th...,Winter Park,FL,US


In [24]:
jobs_US_base_line['Title'] = jobs_US_base_line['Title'].fillna('')
jobs_US_base_line['Description'] = jobs_US_base_line['Description'].fillna('')
#jobs_US_base_line['Requirements'] = jobs_US_base_line['Requirements'].fillna('')

jobs_US_base_line['Description'] = jobs_US_base_line['Title'] + jobs_US_base_line['Description']

In [37]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(jobs_US_base_line['Description'])
tfidf_matrix

<9831x503836 sparse matrix of type '<class 'numpy.float64'>'
	with 2307866 stored elements in Compressed Sparse Row format>

In [26]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [98]:
jobs_US_base_line = jobs_US_base_line.reset_index()
titles = jobs_US_base_line['Title']
indices = pd.Series(jobs_US_base_line.index, index=jobs_US_base_line['Title'])
indices

Title
Security Engineer/Technical Lead                                 0
SAP Business Analyst / WM                                        1
P/T HUMAN RESOURCES ASSISTANT                                    2
Route Delivery Drivers                                           3
Housekeeping                                                     4
SALON/SPA COORDINATOR                                            5
SUPERINTENDENT                                                   6
ELECTRONIC PRE-PRESS PROFESSIONAL                                7
UTILITY LINE TRUCK OPERATOR/ DIGGER DERRICK                      8
CONSTRUCTION PROJECT MGR & PM TRAINEE                            9
Administrative Assistant                                        10
ACCOUNT EXECUTIVES                                              11
COMMERCIAL ESTIMATOR                                            12
Immediate Opening                                               13
TESL Adjunct                                            

In [28]:
def get_recommendations(title):
    idx = indices[title]
    #print (idx)
    sim_scores = list(enumerate(cosine_sim[idx]))
    #print (sim_scores)
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    job_indices = [i[0] for i in sim_scores]
    return titles.iloc[job_indices]

In [29]:
get_recommendations('SAP Business Analyst / WM').head(10)

1                           SAP Business Analyst / WM
5895                    SAP FI/CO Business Consultant
5712                       SAP FI/CO Business Analyst
5004                          SAP Basis Administrator
5196    SAP Sales and Distribution Solution Architect
4641       Senior Specialist - SAP Configuration - SD
4962                       SAP Integration Specialist
5253            Senior Business Systems Analyst - SAP
5089                                 Business Analyst
4573           SAP ABAP Developer with PRA experience
Name: Title, dtype: object

In [38]:
user_based_approach_US = users_training.loc[users_training['Country']=='US']
user_based_approach_US 

,UserID,WindowID,Split,City,State,Country,ZipCode,DegreeType,Major,GraduationDate,WorkHistoryCount,TotalYearsExperience,CurrentlyEmployed,ManagedOthers,ManagedHowMany
0,47,1,Train,Paramount,CA,US,90723,High School,NaN,1999-06-01 00:00:00,3,10.0,Yes,No,0
1,72,1,Train,La Mesa,CA,US,91941,Master's,Anthropology,2011-01-01 00:00:00,10,8.0,Yes,No,0
2,80,1,Train,Williamstown,NJ,US,08094,High School,Not Applicable,1985-06-01 00:00:00,5,11.0,Yes,Yes,5
3,98,1,Train,Astoria,NY,US,11105,Master's,Journalism,2007-05-01 00:00:00,3,3.0,Yes,No,0
4,123,1,Train,Baton Rouge,LA,US,70808,Bachelor's,Agricultural Business,2011-05-01 00:00:00,1,9.0,Yes,No,0
5,131,1,Train,Houston,TX,US,77077,Bachelor's,Finance,1998-05-01 00:00:00,3,14.0,NaN,No,0
6,162,1,Train,Long Beach,CA,US,90807,Master's,I/O Psychology,2012-05-01 00:00:00,10,25.0,No,No,0
7,178,1,Train,Greenville,SC,US,29609,High School,Not Applicable,NaN,6,35.0,No,Yes,4
8,203,1,Train,Colchester,VT,US,05446,Master's,Burlington,NaN,4,3.0,Yes,No,0
9,344,1,Train,Newport News,VA,US,23601,High School,Not Applicable,2007-01-01 00:00:00,3,7.0,Yes,No,0


In [40]:
user_based_approach = user_based_approach_US.iloc[0:10000,:]
user_based_approach = user_based_approach.dropna()
user_based_approach = user_based_approach.reset_index(drop=True)
user_based_approach

,UserID,WindowID,Split,City,State,Country,ZipCode,DegreeType,Major,GraduationDate,WorkHistoryCount,TotalYearsExperience,CurrentlyEmployed,ManagedOthers,ManagedHowMany
0,72,1,Train,La Mesa,CA,US,91941,Master's,Anthropology,2011-01-01 00:00:00,10,8.0,Yes,No,0
1,80,1,Train,Williamstown,NJ,US,08094,High School,Not Applicable,1985-06-01 00:00:00,5,11.0,Yes,Yes,5
2,98,1,Train,Astoria,NY,US,11105,Master's,Journalism,2007-05-01 00:00:00,3,3.0,Yes,No,0
3,123,1,Train,Baton Rouge,LA,US,70808,Bachelor's,Agricultural Business,2011-05-01 00:00:00,1,9.0,Yes,No,0
4,162,1,Train,Long Beach,CA,US,90807,Master's,I/O Psychology,2012-05-01 00:00:00,10,25.0,No,No,0
5,344,1,Train,Newport News,VA,US,23601,High School,Not Applicable,2007-01-01 00:00:00,3,7.0,Yes,No,0
6,437,1,Train,Brookville,OH,US,45309,Associate's,Nursing,2010-05-01 00:00:00,4,15.0,Yes,No,0
7,496,1,Train,Easley,SC,US,29640,High School,Not Applicable,1987-01-01 00:00:00,10,21.0,No,No,0
8,534,1,Train,Nashville,TN,US,37214,High School,Not Applicable,2009-01-01 00:00:00,3,4.0,No,No,0
9,554,1,Train,Altamonte Springs,FL,US,32701,Bachelor's,Legal Studies,2006-12-01 00:00:00,2,4.0,No,No,0


In [44]:
user_based_approach['DegreeType'] = user_based_approach['DegreeType'].fillna('')
user_based_approach['Major'] = user_based_approach['Major'].fillna('')
user_based_approach['TotalYearsExperience'] = str(user_based_approach['TotalYearsExperience'].fillna(''))

user_based_approach['DegreeType'] = user_based_approach['DegreeType'] + user_based_approach['Major'] + user_based_approach['TotalYearsExperience']

In [45]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(user_based_approach['DegreeType'])
tfidf_matrix

<4795x5620 sparse matrix of type '<class 'numpy.float64'>'
	with 681224 stored elements in Compressed Sparse Row format>

In [46]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [148]:

userid = user_based_approach['Major']
global indices
indices = pd.Series(user_based_approach.index, index=user_based_approach['Major'])
#indices.head(2)

In [149]:
def get_recommendations_userwise(uid):
    idx = indices[uid]
    #print (idx)
    sim_scores = list(enumerate(cosine_sim[idx]))
    #print (sim_scores)
    sim_scores = sorted(sim_scores, key=lambda x: x[1].any(), reverse=True)
    user_indices = [i[0] for i in sim_scores]
    #print (user_indices)
    return user_indices[0:11]
def get_job_id(usrid_list):
    jobs_userwise = apps_training['UserID'].isin(usrid_list) #
    df1 = pd.DataFrame(data = apps_training[jobs_userwise], columns=['JobID'])
    joblist = df1['JobID'].tolist()
    Job_list = jobs['JobID'].isin(joblist) #[1083186, 516837, 507614, 754917, 686406, 1058896, 335132])
    df_temp = pd.DataFrame(data = jobs[Job_list], columns=['JobID','Title','Description','City','State'])
    return df_temp

In [158]:
get_job_id(get_recommendations_userwise("Not Applicable"))["Title"]

479106                                               Teller
527966    Customer Service Representative, Now Accepting...
596996                        UPS Part Time Package Handler
657723                          Full Time  Fork Lift Driver
681420                                Now Hiring All Trades
Name: Title, dtype: object

In [159]:
merged=pd.merge(user_based_approach,user_history_training,on="UserID")

In [160]:
merged= merged.iloc[0:10000,:]
merged = merged.dropna()
merged = merged.reset_index(drop=True)
merged

,level_0,index,UserID,WindowID_x,Split_x,City,State,Country,ZipCode,DegreeType,...,GraduationDate,WorkHistoryCount,TotalYearsExperience,CurrentlyEmployed,ManagedOthers,ManagedHowMany,WindowID_y,Split_y,Sequence,JobTitle
0,0,0,72,1,Train,La Mesa,CA,US,91941,Master'sAnthropology0 8.0\n1 11.0...,...,2011-01-01 00:00:00,10,0 8.0\n1 11.0\n2 3.0\n3 ...,Yes,No,0,1,Train,1,"Lecturer, Department of Anthropology"
1,0,0,72,1,Train,La Mesa,CA,US,91941,Master'sAnthropology0 8.0\n1 11.0...,...,2011-01-01 00:00:00,10,0 8.0\n1 11.0\n2 3.0\n3 ...,Yes,No,0,1,Train,2,Student Assistant
2,0,0,72,1,Train,La Mesa,CA,US,91941,Master'sAnthropology0 8.0\n1 11.0...,...,2011-01-01 00:00:00,10,0 8.0\n1 11.0\n2 3.0\n3 ...,Yes,No,0,1,Train,3,Elderly Caregiver
3,0,0,72,1,Train,La Mesa,CA,US,91941,Master'sAnthropology0 8.0\n1 11.0...,...,2011-01-01 00:00:00,10,0 8.0\n1 11.0\n2 3.0\n3 ...,Yes,No,0,1,Train,5,"Department Assistant, Department"
4,0,0,72,1,Train,La Mesa,CA,US,91941,Master'sAnthropology0 8.0\n1 11.0...,...,2011-01-01 00:00:00,10,0 8.0\n1 11.0\n2 3.0\n3 ...,Yes,No,0,1,Train,6,Book Reviews Editorial Assistant
5,0,0,72,1,Train,La Mesa,CA,US,91941,Master'sAnthropology0 8.0\n1 11.0...,...,2011-01-01 00:00:00,10,0 8.0\n1 11.0\n2 3.0\n3 ...,Yes,No,0,1,Train,7,Graduate Assistant
6,0,0,72,1,Train,La Mesa,CA,US,91941,Master'sAnthropology0 8.0\n1 11.0...,...,2011-01-01 00:00:00,10,0 8.0\n1 11.0\n2 3.0\n3 ...,Yes,No,0,1,Train,9,Research Assistant
7,0,0,72,1,Train,La Mesa,CA,US,91941,Master'sAnthropology0 8.0\n1 11.0...,...,2011-01-01 00:00:00,10,0 8.0\n1 11.0\n2 3.0\n3 ...,Yes,No,0,1,Train,10,"Docent, C.E. Smith Museum of Anthropology, Cal..."
8,1,1,80,1,Train,Williamstown,NJ,US,08094,High SchoolNot Applicable0 8.0\n1 ...,...,1985-06-01 00:00:00,5,0 8.0\n1 11.0\n2 3.0\n3 ...,Yes,Yes,5,1,Train,1,"Auto Publishing/Electro Mechanical Technician,..."
9,1,1,80,1,Train,Williamstown,NJ,US,08094,High SchoolNot Applicable0 8.0\n1 ...,...,1985-06-01 00:00:00,5,0 8.0\n1 11.0\n2 3.0\n3 ...,Yes,Yes,5,1,Train,2,"Enhanced Baker Cell Technician, EBC Technician"


In [161]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(merged['JobTitle'])
tfidf_matrix

<9240x12745 sparse matrix of type '<class 'numpy.float64'>'
	with 41838 stored elements in Compressed Sparse Row format>

In [162]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [163]:

indices = pd.Series(merged.index, index=merged['Major'])



In [131]:
def get_recommendations_userwise(userid):
    idx = indices[userid]
    #print (idx)
    sim_scores = list(enumerate(cosine_sim[idx]))
    #print (sim_scores)
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    user_indices = [i[0] for i in sim_scores]
    #print (user_indices)
    return user_indices

In [132]:
get_job_id(get_recommendations_userwise("Psychology"))


KeyError: 'Psychology'

In [167]:
def get_recommendations_userwise(uid):
    idx = indices[uid]
    #print (idx)
    sim_scores = list(enumerate(cosine_sim[idx]))
    #print (sim_scores)
    sim_scores = sorted(sim_scores, key=lambda x: x[1].any(), reverse=True)
    user_indices = [i[0] for i in sim_scores]
    #print (user_indices)
    return user_indices[0:11]

def get_job_id(usrid_list):
    jobs_userwise = apps_training['UserID'].isin(usrid_list) #
    df1 = pd.DataFrame(data = apps_training[jobs_userwise], columns=['JobID'])
    joblist = df1['JobID'].tolist()
    Job_list = jobs['JobID'].isin(joblist) #[1083186, 516837, 507614, 754917, 686406, 1058896, 335132])
    df_temp = pd.DataFrame(data = jobs[Job_list], columns=['JobID','Title','Description','City','State'])
    return df_temp

In [166]:
get_job_id(get_recommendations_userwise("Anthropology"))["Title"]

479106                                               Teller
527966    Customer Service Representative, Now Accepting...
596996                        UPS Part Time Package Handler
657723                          Full Time  Fork Lift Driver
681420                                Now Hiring All Trades
Name: Title, dtype: object